In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

### Import data and pre-processing
* Note that I have skimmed at the data format thus I pre-process accordingly.

In [3]:
def xlsx_preprocess(df, date_format):
    imp = pd.read_excel(df, index_col = 0, header = 0)
    imp.index = pd.to_datetime(imp.index, format = date_format)
    imp.index = imp.index.to_period('M')
    return imp

data =  xlsx_preprocess('Industry_Portfolios.xlsx', '%Y%m')
data.head()

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
2004-01,0.06,-1.07,-0.62,0.44,4.53,1.41,0.45,3.09,1.92,2.88
2004-02,4.25,-0.07,1.95,4.69,-2.92,-0.52,6.09,0.89,2.07,2.16
2004-03,-0.09,-1.15,-0.27,-0.13,-2.55,-2.07,0.29,-3.96,1.13,-0.63
2004-04,1.42,2.30,-0.17,2.52,-4.91,-0.48,-2.70,3.54,-3.55,-3.76
2004-05,-1.89,-1.64,1.61,0.39,4.85,-2.95,0.30,-0.42,1.28,1.86


* Data is present as percentages already. e.g. 0.06 = 0.06%. Thus we may need to convert in the following parts.

### Mean Returns, Standard Deviation and CAGR

In [5]:
def return_sd(r: pd.Series, periods_per_year: int):
    """
    Calculate annualized return, CAGR, and annualized standard deviation (volatility) for a single asset.

    Parameters:
    - r: A pandas Series of returns in decimal format.
    - periods_per_year: The number of periods in a year (e.g., 12 for monthly, 252 for daily).

    Returns:
    - A dictionary of metrics, rounded to 4 decimals.
    """
    obs = len(r)

    # Calculate mean and annualized return
    mean_return = r.mean()
    annualized_return = mean_return * periods_per_year

    # Calculate standard deviation and annualized standard deviation
    sd = r.std()
    annualized_sd = sd * np.sqrt(periods_per_year)

    # Calculate CAGR
    hpr = (1 + r).prod()  # Holding Period Return
    cagr = hpr**(periods_per_year/obs) - 1

    # Return the results as a dictionary with rounded values
    return {
        'Return': round(mean_return, 6),
        'Annualized Return': round(annualized_return, 6),
        'CAGR': round(cagr, 6),
        'Standard deviation': round(sd, 6),
        'Annualized Standard deviation': round(annualized_sd, 6)
    }

In [7]:
return_sd(data.NoDur/100, periods_per_year = 12)

{'Return': 0.009028,
 'Annualized Return': 0.10834,
 'CAGR': 0.106463,
 'Standard deviation': 0.033457,
 'Annualized Standard deviation': 0.115897}

In [9]:
def return_sd_multiple(returns_df: pd.DataFrame, periods_per_year: int):
    """
    Calculate annualized return, CAGR, and annualized standard deviation for multiple assets.
    
    Parameters:
    - returns_df: A DataFrame where each column is a time series of returns for a different asset.
    - periods_per_year: The number of periods in a year (e.g., 12 for monthly, 252 for daily).
    
    Returns:
    - A DataFrame with the metrics for each asset.
    """
    results = []

    # Loop over each column (asset) in the DataFrame
    for asset in returns_df.columns:
        metrics = return_sd(returns_df[asset], periods_per_year)
        metrics['Asset'] = asset  # Add the asset name for reference
        results.append(metrics)

    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    
    # Set the asset names as the index
    results_df.set_index('Asset', inplace=True)
    
    return results_df

In [11]:
return_sd_multiple(data/100, 12)

,Return,Annualized Return,CAGR,Standard deviation,Annualized Standard deviation
Asset,,,,,
NoDur,0.009028,0.10834,0.106463,0.033457,0.115897
Durbl,0.007333,0.08800,0.047788,0.083619,0.289663
Manuf,0.010128,0.12154,0.109516,0.053103,0.183953
Enrgy,0.012312,0.14774,0.132951,0.060815,0.210670
HiTec,0.007662,0.09195,0.077014,0.053812,0.186410
Telcm,0.008814,0.10577,0.097903,0.044483,0.154093
Shops,0.009163,0.10996,0.104609,0.040938,0.141813
Hlth,0.007838,0.09406,0.088937,0.037872,0.131191
Utils,0.009072,0.10886,0.105254,0.037018,0.128233


### Max Drawdown and Calmar ratios

In [13]:
def drawdown(return_series: pd.Series):
    """
    Calculate the drawdown for a single asset's return series.
    
    Parameters:
    - return_series: A pandas Series representing the asset's return series.
    
    Returns:
    - A DataFrame with the drawdown values.
    """
    wealth_index = 1000 * (1 + return_series).cumprod()  # Starting wealth of 1000
    previous_peak = wealth_index.cummax()
    drawdown = (wealth_index - previous_peak) / previous_peak
    
    df = pd.DataFrame({
        'Wealth': wealth_index,
        'Peaks': previous_peak,
        'Drawdowns': drawdown})
    
    return df[['Drawdowns']]  # Return only the Drawdowns column

In [15]:
drawdown(data.NoDur/100)

,Drawdowns
Date,
2004-01,0.000000
2004-02,0.000000
2004-03,-0.000900
2004-04,0.000000
2004-05,-0.018900
...,...
2013-08,-0.040000
2013-09,-0.021376
2013-10,0.000000


In [17]:
def drawdown_multiple_assets(returns_df: pd.DataFrame):
    """
    Calculate the drawdowns for multiple assets' return series.
    
    Parameters:
    - returns_df: A pandas DataFrame where each column represents a return series for an asset.
    
    Returns:
    - A DataFrame containing the drawdowns for each asset.
    """
    # DataFrame to store the results
    results_df = pd.DataFrame()

    # Loop over each asset in the returns DataFrame
    for asset in returns_df.columns:
        return_series = returns_df[asset].dropna()  # Get the return series for the asset
        
        # Calculate drawdown for the asset using the single asset drawdown function
        asset_drawdown_df = drawdown(return_series)
        
        # Rename the 'Drawdowns' column to reflect the asset name
        asset_drawdown_df.columns = [f'{asset} Drawdowns']
        
        # Concatenate the asset's drawdown data with the overall results DataFrame
        results_df = pd.concat([results_df, asset_drawdown_df], axis=1)
    
    return results_df

In [21]:
drawdown_multiple_assets(data/100).tail(1)

,NoDur Drawdowns,Durbl Drawdowns,Manuf Drawdowns,Enrgy Drawdowns,HiTec Drawdowns,Telcm Drawdowns,Shops Drawdowns,Hlth Drawdowns,Utils Drawdowns,Other Drawdowns
Date,,,,,,,,,,
2013-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.022967,-0.012966


In [23]:
def calmar_ratio(return_series: pd.Series, periods_per_year: int):
    """
    Calculate the Calmar Ratio, CAGR, and Maximum Drawdown for a single asset.

    Parameters:
    - return_series: A pandas Series of returns in decimal format.
    - periods_per_year: The number of periods in a year (e.g., 12 for monthly, 252 for daily).

    Returns:
    - A dictionary with CAGR, Maximum Drawdown, and Calmar Ratio.
    """
    # Get the CAGR using the existing return_sd function
    cagr = return_sd(return_series, periods_per_year)['CAGR']
    
    # Get the maximum drawdown from the drawdown function
    drawdown_df = drawdown(return_series)
    max_drawdown = drawdown_df['Drawdowns'].min()  # Minimum drawdown is the maximum drawdown (most negative value)
    
    # Calculate the Calmar Ratio (CAGR / Max Drawdown)
    if max_drawdown == 0:
        calmar_ratio_value = np.nan  # To avoid division by zero
    else:
        calmar_ratio_value = cagr / abs(max_drawdown)  # Take the absolute value of the max drawdown
    
    # Return the results as a dictionary
    return {
        'CAGR': cagr,
        'Max Drawdown': round(max_drawdown, 6),
        'Calmar Ratio': round(calmar_ratio_value, 6)
    }

In [25]:
calmar_ratio(data.NoDur/100, 12)

{'CAGR': 0.106463, 'Max Drawdown': -0.342321, 'Calmar Ratio': 0.311003}

In [27]:
def calmar_ratio_multiple(returns_df: pd.DataFrame, periods_per_year: int):
    """
    Calculate the Calmar Ratio, CAGR, and Maximum Drawdown for multiple assets.

    Parameters:
    - returns_df: A pandas DataFrame where each column is a return series for a different asset.
    - periods_per_year: The number of periods in a year (e.g., 12 for monthly, 252 for daily).

    Returns:
    - A DataFrame with the Calmar Ratio, CAGR, and Maximum Drawdown for each asset.
    """
    # DataFrame to store the results
    results = []

    # Loop over each asset in the returns DataFrame
    for asset in returns_df.columns:
        return_series = returns_df[asset].dropna()  # Get the return series for the asset
        
        # Calculate the Calmar Ratio, CAGR, and Max Drawdown for the asset
        calmar_metrics = calmar_ratio(return_series, periods_per_year)
        
        # Append the results to the list
        calmar_metrics['Asset'] = asset
        results.append(calmar_metrics)
    
    # Convert the results into a DataFrame
    results_df = pd.DataFrame(results)
    
    # Set the asset names as the index
    results_df.set_index('Asset', inplace=True)
    
    return results_df

In [29]:
calmar_ratio_multiple(data/100, 12)

,CAGR,Max Drawdown,Calmar Ratio
Asset,,,
NoDur,0.106463,-0.342321,0.311003
Durbl,0.047788,-0.730457,0.065422
Manuf,0.109516,-0.541242,0.202342
Enrgy,0.132951,-0.501487,0.265113
HiTec,0.077014,-0.500923,0.153744
Telcm,0.097903,-0.519102,0.188601
Shops,0.104609,-0.381394,0.274281
Hlth,0.088937,-0.304307,0.292261
Utils,0.105254,-0.383804,0.274239


### Sharpe, Treynor, Information, Sortino, Semi-deviation, Downside Risk

#### You can do some data manipulations to have the DataFrame similar to this to run the below function

In [31]:
data2 = data.copy()/100
data2['Rf'] = 0.0011 # assumed Rf = 0.11% per month
data2['Benchmark'] = 0.0085  # Assuming the target return is 0.85% per month
data2.head(3)

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Rf,Benchmark
Date,,,,,,,,,,,,
2004-01,0.0006,-0.0107,-0.0062,0.0044,0.0453,0.0141,0.0045,0.0309,0.0192,0.0288,0.0011,0.0085
2004-02,0.0425,-0.0007,0.0195,0.0469,-0.0292,-0.0052,0.0609,0.0089,0.0207,0.0216,0.0011,0.0085
2004-03,-0.0009,-0.0115,-0.0027,-0.0013,-0.0255,-0.0207,0.0029,-0.0396,0.0113,-0.0063,0.0011,0.0085


In [33]:
def performance_ratios_single(data: pd.DataFrame, asset: str, benchmark: str, rf_column: str, beta: float):
    """
    data: DataFrame that contains asset returns, benchmark returns, and risk-free rate.
    asset: The column name of the asset returns.
    benchmark: The column name of the benchmark returns.
    rf_column: The column name of the risk-free rate.
    beta: The CAPM beta for the asset.
    
    Returns a dictionary containing the Sharpe ratio, Sortino ratio, Treynor ratio, Information ratio, and downside risk.
    """
    # Calculate excess returns over the risk-free rate
    excess_return_rf = data[asset] - data[rf_column]

    # Calculate Semi-deviation (below mean return)
    excess_mean = data[asset] - (data[asset].mean())
    below_mean =  excess_mean[excess_mean < 0]
    n_below_mean = len(below_mean)
    semi_var = (np.sum(below_mean**2)) / n_below_mean
    semi_dev = semi_var**0.5
    
    # Sharpe Ratio: (mean return - rf) / std deviation of returns
    sharpe_ratio = excess_return_rf.mean() / excess_return_rf.std()
    
    # Treynor Ratio: (mean return - rf) / beta
    treynor_ratio = excess_return_rf.mean() / beta
    
    # Information Ratio: (mean return - benchmark return) / tracking error
    excess_return_benchmark = data[asset] - data[benchmark]
    tracking_error = excess_return_benchmark.std()
    information_ratio = (excess_return_benchmark.mean()) / tracking_error if tracking_error != 0 else np.nan

    # Sortino Ratio: (mean return - benchmark return) / below-benchmark semi-deviation (note divided by n or n - 1)
    below_benchmark = excess_return_benchmark[excess_return_benchmark < 0]
    downside_var = (np.sum(below_benchmark**2)) / (len(data) - 1)
    downside_risk =  downside_var**0.5
    sortino_ratio = (excess_return_benchmark.mean()) / downside_risk if downside_risk != 0 else np.nan
    
    # Return all the calculated ratios and downside risk
    return {
        'Semi Deviation': round(semi_dev, 4),
        'Sharpe Ratio': round(sharpe_ratio, 4),
        'Sortino Ratio': round(sortino_ratio, 4),
        'Treynor Ratio': round(treynor_ratio, 4),
        'Information Ratio': round(information_ratio, 4),
        'Downside Risk': round(downside_risk, 4)}

# Note again: the only difference between Information Ratio and Sortino Ratio is the denominator.
# The denominator in downside_risk is divided by n or n - 1, not the number of returns below target, otherwise biased.

In [35]:
# Assuming the target return is 0.85% per month and the asset beta is 1.3
performance_ratios_single(data2, 'NoDur', 'Benchmark', 'Rf', 1.3)

{'Semi Deviation': 0.0381,
 'Sharpe Ratio': 0.237,
 'Sortino Ratio': 0.0206,
 'Treynor Ratio': 0.0061,
 'Information Ratio': 0.0158,
 'Downside Risk': 0.0256}

In [37]:
def performance_ratios_multiple(data: pd.DataFrame, assets: list, benchmark: str, rf_column: str, betas: dict):
    """
    Calculate performance ratios for multiple assets.
    
    Parameters:
    - data: DataFrame that contains asset returns, benchmark returns, and risk-free rate.
    - assets: List of asset columns to calculate the performance ratios for.
    - benchmark: The column name of the benchmark returns.
    - rf_column: The column name of the risk-free rate.
    - betas: A dictionary containing beta values for each asset.
    
    Returns a DataFrame containing the performance ratios for each asset.
    """
    results = []

    # Loop over each asset in the asset list
    for asset in assets:
        if asset in betas:
            # Calculate the performance ratios for the asset
            metrics = performance_ratios_single(data, asset, benchmark, rf_column, betas[asset])
            metrics['Asset'] = asset  # Add asset name for reference
            results.append(metrics)
    
    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    
    # Set the asset name as the index
    results_df.set_index('Asset', inplace=True)
    
    return results_df

In [39]:
betas = {
    'NoDur': 1.3,
    'Durbl': 0.7,
    'Manuf': 1.2,
    'Enrgy': 0.88,
    'Telcm': 1.25,
    'Shops': 1.3,
    'Hlth': 0.78,
    'Utils': 0.77,
    'Other': 1.4
} # Assumption

In [41]:
assets = list(betas.keys())
assets

['NoDur',
 'Durbl',
 'Manuf',
 'Enrgy',
 'Telcm',
 'Shops',
 'Hlth',
 'Utils',
 'Other']

In [43]:
performance_ratios_multiple(data2, assets, 'Benchmark', 'Rf', betas)

,Semi Deviation,Sharpe Ratio,Sortino Ratio,Treynor Ratio,Information Ratio,Downside Risk
Asset,,,,,,
NoDur,0.0381,0.2370,0.0206,0.0061,0.0158,0.0256
Durbl,0.0815,0.0745,-0.0201,0.0089,-0.0140,0.0579
Manuf,0.0613,0.1700,0.0406,0.0075,0.0307,0.0402
Enrgy,0.0681,0.1844,0.0867,0.0127,0.0627,0.0440
Telcm,0.0513,0.1734,0.0091,0.0062,0.0071,0.0344
Shops,0.0452,0.1970,0.0218,0.0062,0.0162,0.0304
Hlth,0.0406,0.1779,-0.0235,0.0086,-0.0175,0.0282
Utils,0.0493,0.2153,0.0190,0.0104,0.0154,0.0301
Other,0.0676,0.0679,-0.0801,0.0027,-0.0647,0.0451
